
# CPython, Python e sua aplicação de alto desempenho
**Python Brasil 2025 — Esbel Tomas Valero Orellana**

> Este notebook acompanha a palestra e contém o conteúdo dos slides com exemplos executáveis.
>
> **Objetivo**: mostrar como combinar Python com C/C++ (via CPython) e explorar paralelismo (OpenMP / CUDA) para acelerar aplicações, usando o problema de multiplicação de matrizes como estudo de caso.

## Sumário
1. [Preparação do ambiente](#prep)
2. [Baseline 1: Python puro (loops)](#py-puro)
3. [Baseline 2: NumPy (BLAS/LAPACK)](#numpy)
4. [Comparativos de desempenho (gráfico)](#comparativo)
5. [Extensão nativa com CPython API (C)](#cpython)
6. [OpenMP (CPU)](#openmp)
7. [CUDA / GPU (opcional)](#cuda)
8. [Discussão e próximos passos](#fim)

> **Aviso**: alguns experimentos podem levar tempo, dependendo do tamanho da matriz. Use o modo *rápido* para testes durante a apresentação.



<a id="prep"></a>

## 1) Preparação do ambiente

- Este notebook usa apenas **NumPy** e **Matplotlib** (e, opcionalmente, **CuPy** se houver GPU).
- Para a parte de CPython (C), geramos arquivos `mymatmul.c` e `setup.py` e compilamos localmente.
- Ajuste o modo **QUICK** para reduzir o tempo dos testes.


In [ ]:

# Configurações gerais
import sys, platform, time, math, statistics as stats
import numpy as np

try:
    import matplotlib.pyplot as plt
except Exception as e:
    raise SystemExit("Matplotlib é necessário para os gráficos. Instale e reexecute.")

# Reprodutibilidade
np.random.seed(42)

# Modo rápido (True: matrizes menores; False: matrizes maiores)
QUICK = True

# Tamanhos padrão (ajuste ao seu hardware)
SIZES = [64, 128, 256] if QUICK else [128, 256, 512, 1024]
REPEATS = 3 if QUICK else 5

print(f"Python {sys.version.split()[0]} | NumPy {np.__version__}")
print(f"Plataforma: {platform.platform()}")
print("Modo QUICK:", QUICK)
print("Tamanhos:", SIZES, "| Repetições por ponto:", REPEATS)



<a id="py-puro"></a>

## 2) Baseline 1 — Multiplicação de matrizes em **Python puro**

Propósito: visualizar o custo de loops aninhados em Python (sob o CPython e o GIL).
**Atenção**: para tamanhos grandes, este método é muito lento — mantenha tamanhos pequenos no modo *rápido*.


In [ ]:

def matmul_py(A, B):
    """
    Multiplicação de matrizes (listas de listas) 3 loops em Python puro.
    A e B devem ser listas de listas de float (ou ints).
    """
    n = len(A)
    m = len(B[0])
    p = len(B)
    C = [[0.0]*m for _ in range(n)]
    for i in range(n):
        Ai = A[i]
        Ci = C[i]
        for k in range(p):
            aik = Ai[k]
            Bk = B[k]
            for j in range(m):
                Ci[j] += aik * Bk[j]
    return C

# Teste pequeno para validar a função
n = 32
A_small = (np.random.rand(n, n)).tolist()
B_small = (np.random.rand(n, n)).tolist()
C_small = matmul_py(A_small, B_small)
print(f"OK: C_small shape = {len(C_small)}x{len(C_small[0])}")


In [ ]:

# Benchmark rápido Python puro vs NumPy (pequenos)
n = 64 if QUICK else 128
A = np.random.rand(n, n)
B = np.random.rand(n, n)

t0 = time.time()
_ = matmul_py(A.tolist(), B.tolist())
t_py = time.time() - t0

t0 = time.time()
_ = A @ B
t_np = time.time() - t0

print(f"Tamanho {n}x{n} | Python puro: {t_py:.3f}s | NumPy: {t_np:.3f}s | Razão (py/np): {t_py/t_np:.1f}×")



<a id="numpy"></a>

## 3) Baseline 2 — **NumPy** (BLAS/LAPACK por baixo dos panos)

- `A @ B` chama rotinas otimizadas em C/Fortran (BLAS).
- Para muitos casos, **isso já basta** e é a melhor relação esforço/ganho.


In [ ]:

def bench_numpy(sizes=SIZES, repeats=REPEATS, dtype=np.float64):
    results = []
    for n in sizes:
        times = []
        for _ in range(repeats):
            A = np.random.rand(n, n).astype(dtype, copy=False)
            B = np.random.rand(n, n).astype(dtype, copy=False)
            t0 = time.time()
            C = A @ B
            times.append(time.time() - t0)
        results.append({"n": n, "mean": float(np.mean(times)), "std": float(np.std(times)), "dtype": str(dtype)})
    return results

numpy_results = bench_numpy()
numpy_results



<a id="comparativo"></a>

## 4) Comparativo de desempenho (gráfico)

- Gráfico do tempo (s) em função do tamanho da matriz `n`.
- Inclui Python puro **apenas para um tamanho de referência**.


In [ ]:

# Coletar um ponto de Python puro para o menor n
n_ref = SIZES[0]
A = np.random.rand(n_ref, n_ref)
B = np.random.rand(n_ref, n_ref)
t0 = time.time()
_ = matmul_py(A.tolist(), B.tolist())
py_point = (n_ref, time.time() - t0)

# Preparar dados NumPy
xs = [r["n"] for r in numpy_results]
ys = [r["mean"] for r in numpy_results]

# Plot (uma figura, sem cores específicas)
plt.figure(figsize=(6,4))
plt.plot(xs, ys, marker="o", label="NumPy (@)")
plt.scatter([py_point[0]], [py_point[1]], marker="x", s=80, label=f"Python puro (n={n_ref})")
plt.xlabel("Tamanho n (n×n)")
plt.ylabel("Tempo (s)")
plt.title("Multiplicação de matrizes: tempo × tamanho")
plt.grid(True, linestyle="--", alpha=0.5)
plt.legend()
plt.show()



<a id="cpython"></a>

## 5) Extensão nativa com **CPython API (C)**

Nesta seção, criamos um módulo C chamado `mymatmul` que expõe `matmul_omp(A, B)`:
- Recebe **arrays NumPy** (contíguos, `float64`).
- Usa a C-API do NumPy para acessar memória bruta.
- Implementa multiplicação em C com laços otimizados e, opcionalmente, **OpenMP**.

> Compilar extensões nativas depende do seu ambiente (compilador, flags, etc.).


In [ ]:

# Gerar mymatmul.c
mymatmul_c = r'''
#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION
#include <Python.h>
#include <numpy/arrayobject.h>

#ifdef _OPENMP
#include <omp.h>
#endif

// C = A @ B, A (n x p), B (p x m), double (contíguo)
static PyObject* matmul_omp(PyObject* self, PyObject* args) {
    PyObject *Aobj, *Bobj;
    if (!PyArg_ParseTuple(args, "OO", &Aobj, &Bobj)) {
        return NULL;
    }

    PyArrayObject *A = (PyArrayObject*) PyArray_FROM_OTF(Aobj, NPY_DOUBLE, NPY_ARRAY_IN_ARRAY);
    PyArrayObject *B = (PyArrayObject*) PyArray_FROM_OTF(Bobj, NPY_DOUBLE, NPY_ARRAY_IN_ARRAY);
    if (!A || !B) {
        Py_XDECREF(A); Py_XDECREF(B);
        PyErr_SetString(PyExc_TypeError, "Falha ao converter inputs para NumPy arrays (float64 contíguos).");
        return NULL;
    }

    if (PyArray_NDIM(A) != 2 || PyArray_NDIM(B) != 2) {
        Py_DECREF(A); Py_DECREF(B);
        PyErr_SetString(PyExc_ValueError, "A e B devem ser matrizes 2D.");
        return NULL;
    }

    npy_intp n = PyArray_DIM(A, 0);
    npy_intp p = PyArray_DIM(A, 1);
    npy_intp p2 = PyArray_DIM(B, 0);
    npy_intp m = PyArray_DIM(B, 1);
    if (p != p2) {
        Py_DECREF(A); Py_DECREF(B);
        PyErr_SetString(PyExc_ValueError, "Dimensões incompatíveis: A(n×p) e B(p×m).");
        return NULL;
    }

    npy_intp dims[2] = {n, m};
    PyArrayObject *C = (PyArrayObject*) PyArray_ZEROS(2, dims, NPY_DOUBLE, 0);
    if (!C) {
        Py_DECREF(A); Py_DECREF(B);
        PyErr_NoMemory();
        return NULL;
    }

    double *a = (double*) PyArray_DATA(A);
    double *b = (double*) PyArray_DATA(B);
    double *c = (double*) PyArray_DATA(C);

    npy_intp lda = PyArray_STRIDES(A)[0] / sizeof(double);
    npy_intp ldb = PyArray_STRIDES(B)[0] / sizeof(double);
    npy_intp ldc = PyArray_STRIDES(C)[0] / sizeof(double);

    #pragma omp parallel for if(n*m > 16384) schedule(static)
    for (npy_intp i = 0; i < n; ++i) {
        for (npy_intp k = 0; k < p; ++k) {
            double aik = a[i*lda + k];
            const double* bk = b + k*ldb;
            double* ci = c + i*ldc;
            for (npy_intp j = 0; j < m; ++j) {
                ci[j] += aik * bk[j];
            }
        }
    }

    Py_DECREF(A); Py_DECREF(B);
    return (PyObject*) C;
}

static PyMethodDef Methods[] = {
    {"matmul_omp", matmul_omp, METH_VARARGS, "C = A @ B (double), com OpenMP opcional."},
    {NULL, NULL, 0, NULL}
};

static struct PyModuleDef moduledef = {
    PyModuleDef_HEAD_INIT,
    "mymatmul",
    "Módulo de multiplicação de matrizes em C com OpenMP opcional.",
    -1,
    Methods
};

PyMODINIT_FUNC PyInit_mymatmul(void) {
    import_array();
    return PyModule_Create(&moduledef);
}
'''
open("mymatmul.c", "w", encoding="utf-8").write(mymatmul_c)
print("Arquivo gerado: mymatmul.c")


In [ ]:

# Gerar setup.py
setup_py = r'''
from setuptools import setup, Extension
import sys
import numpy

extra_compile_args = ['-O3']
extra_link_args = []

if sys.platform.startswith('linux') or sys.platform == 'darwin':
    extra_compile_args += ['-fopenmp']
    extra_link_args += ['-fopenmp']

ext = Extension(
    name='mymatmul',
    sources=['mymatmul.c'],
    include_dirs=[numpy.get_include()],
    extra_compile_args=extra_compile_args,
    extra_link_args=extra_link_args,
)

setup(
    name='mymatmul',
    version='0.1.0',
    description='Multiplicação de matrizes em C com OpenMP opcional (CPython API).',
    ext_modules=[ext],
)
'''
open("setup.py", "w", encoding="utf-8").write(setup_py)
print("Arquivo gerado: setup.py")



### Compilação

> Execute **localmente** (pode não funcionar em todos os ambientes online):

```bash
python setup.py build_ext --inplace
```

Se tudo der certo, você verá `mymatmul.*.so` (Linux/Mac) ou `.pyd` (Windows).


In [ ]:

# Teste de importação (funciona somente após compilar)
try:
    import mymatmul
    print("mymatmul importado com sucesso.")
except Exception as e:
    print("mymatmul ainda não está compilado/importável:", e)


In [ ]:

# Benchmark opcional usando a extensão C (se disponível)
def bench_extension(sizes, repeats):
    import mymatmul
    res = []
    for n in sizes:
        times = []
        for _ in range(repeats):
            A = np.random.rand(n, n).astype(np.float64, copy=False)
            B = np.random.rand(n, n).astype(np.float64, copy=False)
            t0 = time.time()
            C = mymatmul.matmul_omp(A, B)
            times.append(time.time() - t0)
        res.append({"n": n, "mean": float(np.mean(times)), "std": float(np.std(times))})
    return res

ext_results = None
try:
    import mymatmul  # noqa
    ext_results = bench_extension(SIZES, REPEATS)
    ext_results
except Exception as e:
    print("Extensão não disponível; pule esta etapa. Motivo:", e)


In [ ]:

# Plot comparativo: NumPy vs Extensão C (se existir)
if 'ext_results' in globals() and ext_results:
    xs_np = [r["n"] for r in numpy_results]
    ys_np = [r["mean"] for r in numpy_results]
    xs_ex = [r["n"] for r in ext_results]
    ys_ex = [r["mean"] for r in ext_results]

    plt.figure(figsize=(6,4))
    plt.plot(xs_np, ys_np, marker="o", label="NumPy (@)")
    plt.plot(xs_ex, ys_ex, marker="s", label="CPython C (OpenMP)")
    plt.xlabel("Tamanho n (n×n)")
    plt.ylabel("Tempo (s)")
    plt.title("Comparativo: NumPy vs Extensão CPython (C/OpenMP)")
    plt.grid(True, linestyle="--", alpha=0.5)
    plt.legend()
    plt.show()
else:
    print("Sem extensão compilada — gráfico comparativo omitido.")



<a id="openmp"></a>

## 6) Nota sobre **OpenMP (CPU)**

- O código C usa `#pragma omp parallel for` (condicional com `_OPENMP`).
- As flags `-fopenmp` (GCC/Clang) habilitam a paralelização.
- Ajuste `OMP_NUM_THREADS` para controlar o número de threads.

Exemplo:
```bash
export OMP_NUM_THREADS=4
python -c "import numpy as np, mymatmul as mm; A=np.random.rand(512,512); B=np.random.rand(512,512); mm.matmul_omp(A,B)"
```



<a id="cuda"></a>

## 7) CUDA / GPU (opcional)

### Opção rápida (sem compilar C/CUDA): **CuPy**
Se você tiver CUDA instalado e uma GPU NVIDIA, a biblioteca **CuPy** replica a API do NumPy e já usa CUDA/CuBLAS.

```python
try:
    import cupy as cp
    n = 1024
    A_gpu = cp.random.rand(n, n, dtype=cp.float64)
    B_gpu = cp.random.rand(n, n, dtype=cp.float64)
    cp.cuda.Stream.null.synchronize()
    t0 = time.time()
    C_gpu = A_gpu @ B_gpu
    cp.cuda.Stream.null.synchronize()
    print("Tempo CuPy:", time.time() - t0)
except Exception as e:
    print("CuPy não disponível:", e)
```

### (Avançado) Kernel CUDA em C
Para quem deseja compilar manualmente um `.cu` e expor via `ctypes`:
1. Escrever um kernel CUDA com `__global__` para `matmul`.
2. Compilar com `nvcc -Xcompiler -fPIC -shared -o libmmcuda.so mm.cu`.
3. Carregar em Python via `ctypes.CDLL("libmmcuda.so")` e escrever *wrappers*.



<a id="fim"></a>

## 8) Discussão e próximos passos

- **Quando vale a pena escrever C/C++?** Partes críticas e *hotspots*.
- **NumPy primeiro**: costuma ser suficiente para 80% dos casos.
- **Extensões CPython**: máximo controle e desempenho, com custo de complexidade.
- **GPU**: use **CuPy** para acelerar com baixa fricção; para casos extremos, CUDA C/C++.

### Referências
- Documentação oficial (Extending/Embedding): https://docs.python.org/3/extending/
- NumPy C-API: https://numpy.org/doc/stable/reference/c-api/
- OpenMP: https://www.openmp.org/
- CuPy: https://cupy.dev/
